In [2]:
from pathlib import Path
import pandas as pd
import json
import pickle

In [3]:
top_300_players_list = pickle.load(open('data/good_players.pkl', 'rb'))
top_300_players_list[:20]

['bfrFniM_byjqe0yt3lzNEqQDfoxTZ1bC5M3gnLgl9T7Yw1M-8E_cnKhDbdN2Eob6C8os3MMQ8Xqh8A',
 'WC05gc3mffI3mNFBP0yEF20n9XnTgHDTmDvUH9tO7xvQBwiyMcERMF28cG6dmoO5Q6bmSfrIFvcBpQ',
 '65u79bsl8sT2n4XpyIssa3NMWwHwVcUZQwakpk27LrRUIGQhQYA2xnL6xUIbJadaFv_IgVmYRYXS_g',
 'b1HcD-_OoW3V_x6iNYTLpeNzfs6ZLnR4wPbLl1ElyRvPvmSZBpjQAYxRv8lEpL80cVqilWZzqFKWvw',
 'tSP3lb9fqt2tPk_0er_EdVvsljH8DXAHhH9slHP_ROLVYN53leD4mf9RT2wDqYRMqswjfEsUoPWqlg',
 '6CWJj7e0A2lcvWGe_h1pMfmXF2egvI6xCFg0eTOV6kEGH9-IBxlZu-bhkjPvNP1lA8Tq6h06nOAVvQ',
 'UsuVlJOUntkcBF-Up7dOSGrGy0VR4jIPN3yzB1LJZIceH3gh-1JVbBQvdEPA1R4JKInzuLK6Xj_ewA',
 'ZqbYXjqBmhtyC_g5XJRasMoHAvdJijYNLORNguY7CVd4aaHmAkcNXH5njkrZ4Y69LUM8OfoISidnfw',
 '5S1Bwr8dVZ3-pEtIcxbxSlsYEP_bDHpnTWqf2ipnVl0xOQ_n06PVwcbuPJrBUU_gLqGYG7pXTg_q_w',
 'WiM9BB14jmARmLfVaUGt2Yf_vd-3lamsvb8azlXZXGZKbCIR_wbOxj1vjaRAtnR3hRPTpTFDMo32Rg',
 't0qHAlMV7RfbavOifUP1q-Xe56lymj169KtY7XdV_Vcje7p6ldVkHnr-MaaoF0n9GEmBM9a7Z7kzfg',
 '-0KPI0YsDEGnRNGB63SsdEHN3JBUyGghfmUw4Iu6veGIKWsClQ9SRwj69ZSHSgvdTl--i4DICNDn2w',
 'Xg

In [4]:
data_path = Path('data/')

In [5]:
summoner_folders = []
for summoner_folder in data_path.iterdir():
    if str(summoner_folder.name).startswith('.') or summoner_folder.name == 'good_players.pkl':
        continue
    print(summoner_folder)
    summoner_folders.append(summoner_folder)

data/SbR3fv6Tq6pC0vS4QTiWOZbb68cPel8aizFRSf5_VZFGR3b5E-a_iK4NfpIwpx8AQSB23sjCFPyvLg
data/lcnE0PmNsvANVZi9jP5vr4cjkJFFCUpFy0fvzZE9dgz0X6Gpx6qlTf0f6RpDPoAibtCwS4ljNq6JZA
data/ck3YMYjZOPl7LRpNiPEpXXCl5EIxKUzOKqzIMbdcH29sY0YRprC5Skh9hactrzqC4KEYGVm6kvZzpg
data/0kp8XNJQjmC9Ws2WCGoLaCdTNivOler67y8ZfI1wQ9aHJDoFTvMHiE2ySwjSlHV_3Sz0kW-MkTayxQ
data/kHdmjAwa4O8cxIuEX9KWTLO2NQhpG7mRjHReJDy1Ya21T5X5xnUEMsnTBnQox71DohXufr0j2Xm1bA
data/kX8IAQUxtudPELGdhgZlSa4GbLgj7B3N3yUGh2VDztLCM83YnfZ8v2x7bf-nAidHkTfkf3xX72qeZQ
data/gtNMsF6mKjWUhTo5-xvdHbUFXiZFHfId6B7krYK24VLAw76pWgtMRJX-fkNAktOn8TAZc5-YNzdOuA
data/e1Xp4409xf1AGCXSDhtdVdE919UHCvN2zUuZjUaenW0bEp1QepTj7zyX0YPEOiQMdmL95fezQwvOyw
data/kHA8bGgUoElP_IkhjzJ-PmhcuZL131t3nyAch1EMUhllSawyLrr0i3J6kPf6sRDF2QaCAC2gnOwP9A
data/RdnsGv2H7Bh3t9chlc5zuC6TPtiYAI2fVW16bKcDnTVO7uvsVsxlNFE7gpBlVj3bD0DiG40IBvv75A
data/PNRO-4SU-ZghuR9fBdyREwAMhqGnZ6UERU03ZdF0VBFYmwugMQED_-ER_fvc-3IIJuETKvEAM1e35g
data/puALT0g5p_VU_0Dkmw0xbpMvJghmqBi7BYrtQJeKvlrQommi4Kqw5umqb1ZSvOUOXctZzTl

In [6]:
import flatdict

def get_all_games_per_user(user_folder):
    participant_one_hot = ['championId', 'individualPosition','role', 'summoner1Id', "summoner2Id"]

    participant_drop = ['perks','challenges','championName', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'lane', 'puuid', 'riotIdName', 'riotIdTagline', 'summonerId', 'summonerName', 'teamEarlySurrendered', 'teamId', 'teamPosition']
    
    
    all_games_df = pd.DataFrame()
    for game_file in user_folder.iterdir():
        print(game_file.name)
        if not game_file.name.startswith('NA1_'):
            continue
        user_id = user_folder.name.split('.')[0]
        
        in_top_300 = user_id in top_300_players_list
        with open(game_file, 'r') as f:
            game_json = json.load(f)

            game_info_dict = {
                'user_id': user_id,
                'in_top_300': in_top_300
            }
            metadata = game_json['metadata']
            game_info_dict = {**game_info_dict, **metadata}
            game_info_dict['num_participants'] = len(metadata['participants']) 
            game_info = game_json['info']

            
            team_id_won = -1
            for team in game_info['teams']:
                if team['win'] == True:
                    team_id_won = team['teamId']
            game_info_dict['team_won'] = team_id_won
            
            user_id_index = metadata['participants'].index(user_id)
            for i, participant_info in enumerate(game_info['participants']):
                for col in participant_drop:
                    if col in participant_info:
                        del participant_info[col]
                participant_info_flattened_orig = flatdict.FlatDict(participant_info, delimiter='.')
                participant_info_flattened = {f'participant{i}_{k}':v for (k,v) in participant_info_flattened_orig.items()}
                
                game_info_dict = {**game_info_dict, **participant_info_flattened}
                if user_id_index == i:
                    my_info = {f'participant_me_{k}':v for (k,v) in participant_info_flattened_orig.items()}
                    game_info_dict = {**game_info_dict, **my_info}
                
            game_info_dict['primary_part_index'] = user_id_index
            del game_info['participants']
            game_info_dict = {**game_info_dict, **game_info}

            game_info_df = pd.DataFrame([game_info_dict])

            all_games_df = pd.concat([all_games_df, game_info_df])
    return all_games_df.reset_index(drop=True)

In [7]:
all_users_df = pd.DataFrame()

for user_path in summoner_folders:
    print(user_path)
    all_users_df = pd.concat([all_users_df, get_all_games_per_user(user_path)])
# user_path = Path('data/VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw')
# print(user_path)
# all_users_df = pd.concat([all_users_df, get_all_games_per_user(user_path)])
all_users_df

data/SbR3fv6Tq6pC0vS4QTiWOZbb68cPel8aizFRSf5_VZFGR3b5E-a_iK4NfpIwpx8AQSB23sjCFPyvLg
NA1_4287589467.json
NA1_4300589612.json
NA1_4290881794.json
NA1_4292866609.json
NA1_4293694040.json
NA1_4301514909.json
NA1_4300571211.json
NA1_4287740160.json
NA1_4300543197.json
NA1_4301540842.json
NA1_4287627795.json
NA1_4292792468.json
NA1_4300496454.json
NA1_4290766654.json
NA1_4300473258.json
NA1_4293764354.json
NA1_4293708690.json
NA1_4293618526.json
NA1_4292798668.json
NA1_4295872177.json
data/lcnE0PmNsvANVZi9jP5vr4cjkJFFCUpFy0fvzZE9dgz0X6Gpx6qlTf0f6RpDPoAibtCwS4ljNq6JZA
NA1_4297549432.json
NA1_4298429458.json
NA1_4298288008.json
NA1_4298351225.json
NA1_4297632271.json
NA1_4303557049.json
NA1_4303741438.json
NA1_4303706500.json
NA1_4298414950.json
NA1_4303691072.json
NA1_4300369925.json
NA1_4297628684.json
NA1_4300285736.json
NA1_4298537370.json
NA1_4298365529.json
NA1_4298502570.json
NA1_4298284669.json
NA1_4303644222.json
NA1_4303355292.json
NA1_4298573967.json
data/ck3YMYjZOPl7LRpNiPEpXXCl5EI

,user_id,in_top_300,dataVersion,matchId,participants,num_participants,team_won,participant0_assists,participant0_baronKills,participant0_bountyLevel,...,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,platformId,queueId,teams,tournamentCode
0,SbR3fv6Tq6pC0vS4QTiWOZbb68cPel8aizFRSf5_VZFGR3...,True,2,NA1_4287589467,[4n6VLbg0pk_cueIdBdSBhlzIeg7m7G8fj8VfcI8P9EZQy...,10,200,0,0,0,...,CLASSIC,teambuilder-match-4287589467,1650843479372,MATCHED_GAME,12.7.433.4138,11,NA1,420,"[{'bans': [{'championId': 555, 'pickTurn': 1},...",
1,SbR3fv6Tq6pC0vS4QTiWOZbb68cPel8aizFRSf5_VZFGR3...,True,2,NA1_4300589612,[kHdmjAwa4O8cxIuEX9KWTLO2NQhpG7mRjHReJDy1Ya21T...,10,100,4,1,1,...,CLASSIC,teambuilder-match-4300589612,1651978529941,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 75, 'pickTurn': 1}, ...",
2,SbR3fv6Tq6pC0vS4QTiWOZbb68cPel8aizFRSf5_VZFGR3...,True,2,NA1_4290881794,[cnwEcRHSgzqksmNOr3BYJWFOvQmRo9JndCbZtmGGkxXyQ...,10,200,2,0,0,...,CLASSIC,teambuilder-match-4290881794,1651113086324,MATCHED_GAME,12.8.436.3750,11,NA1,420,"[{'bans': [{'championId': 555, 'pickTurn': 1},...",
3,SbR3fv6Tq6pC0vS4QTiWOZbb68cPel8aizFRSf5_VZFGR3...,True,2,NA1_4292866609,[xNrMpJ3lnm38urzuG0Dlp6VhEOhhjYWUTmrhLJVb2uwGA...,10,100,7,0,2,...,CLASSIC,teambuilder-match-4292866609,1651287572161,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 555, 'pickTurn': 1},...",
4,SbR3fv6Tq6pC0vS4QTiWOZbb68cPel8aizFRSf5_VZFGR3...,True,2,NA1_4293694040,[hco4DhxnUnErXbsgjwKNQmbZdgyE9Bvrt_vPBx_nzMwQZ...,10,100,2,0,7,...,CLASSIC,teambuilder-match-4293694040,1651366995395,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 17, 'pickTurn': 1}, ...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,BywJ1P_405VjTG0f_2qyGiigo4C0xasV9TbJ4RKJiwRdur...,False,2,NA1_4301433958,[iy-2DoOyYXY91pPmTtrozsaVUu6IocX_6GZz2PSY34-tz...,10,100,1,0,0,...,CLASSIC,teambuilder-match-4301433958,1652057472418,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 350, 'pickTurn': 1},...",
16,BywJ1P_405VjTG0f_2qyGiigo4C0xasV9TbJ4RKJiwRdur...,False,2,NA1_4301975359,[Ugw7D0JJHfMTGPXi9cALq2TZQyzVMCD3irjAKl48vV1-V...,10,100,4,0,0,...,CLASSIC,teambuilder-match-4301975359,1652117262303,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 141, 'pickTurn': 1},...",
17,BywJ1P_405VjTG0f_2qyGiigo4C0xasV9TbJ4RKJiwRdur...,False,2,NA1_4301949055,[C2BITvp74bQl6e05chswKyUlRAfqnr91Cxo7acEuYYy9E...,10,200,3,0,0,...,CLASSIC,teambuilder-match-4301949055,1652114735280,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 30, 'pickTurn': 1}, ...",
18,BywJ1P_405VjTG0f_2qyGiigo4C0xasV9TbJ4RKJiwRdur...,False,2,NA1_4299844569,[A14R34SU7PMdW4Fc8uMsM_mcQlU1Q_41EHWA7RBS1VkW8...,10,100,5,0,3,...,CLASSIC,teambuilder-match-4299844569,1651907097119,MATCHED_GAME,12.8.437.6765,11,NA1,420,"[{'bans': [{'championId': 105, 'pickTurn': 1},...",


In [8]:
all_users_df.reset_index(inplace=True, drop=True)

In [9]:
ml_drop_cols = ['matchId', 'in_top_300', 'dataVersion', 'participants', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'platformId', 'queueId', 'tournamentCode', 'team_won', 'num_participants']           

first_user_filter_df = all_users_df.loc[:, all_users_df.columns.str.startswith('participant_me_') | all_users_df.columns.isin(['user_id'])]
per_user_stats_df = first_user_filter_df.groupby('user_id').mean()
per_user_stats_df

,participant_me_assists,participant_me_baronKills,participant_me_bountyLevel,participant_me_champExperience,participant_me_champLevel,participant_me_championId,participant_me_championTransform,participant_me_consumablesPurchased,participant_me_damageDealtToBuildings,participant_me_damageDealtToObjectives,...,participant_me_trueDamageTaken,participant_me_turretKills,participant_me_turretTakedowns,participant_me_turretsLost,participant_me_unrealKills,participant_me_visionScore,participant_me_visionWardsBoughtInGame,participant_me_wardsKilled,participant_me_wardsPlaced,participant_me_win
user_id,,,,,,,,,,,,,,,,,,,,,
-0KPI0YsDEGnRNGB63SsdEHN3JBUyGghfmUw4Iu6veGIKWsClQ9SRwj69ZSHSgvdTl--i4DICNDn2w,6.75,0.05,0.85,11843.60,13.40,292.05,0.0,4.75,1232.60,4903.85,...,533.40,0.50,1.65,3.55,0.0,22.75,3.75,3.00,9.90,0.60
-8UlI3VoUkbxefvJVM42CFXwVWfWhnpRHzwO7CVSa4AFTiqYO_9yLvMraFWDH-3O0lwKKn1XJFntig,4.85,0.05,1.45,13492.85,14.40,59.45,0.0,3.55,2020.75,4632.80,...,2506.05,0.70,1.50,5.45,0.0,17.95,1.90,1.70,8.20,0.35
-Bb7e_CGHXUBfotoVjioJbBfRTiBYnxbJt2vy12nSh10YlxbnnXjZ-KapGRBsAJZ9wAId6LB6EAXzw,8.45,0.30,0.80,12229.00,13.70,92.65,0.0,3.50,2016.00,10813.40,...,1754.00,0.90,1.60,5.40,0.0,20.55,2.55,2.15,8.90,0.55
-Io51fxPoNj_Lr9Vrjy0Y0E2nUGxd8xklHEuB_G3N5E0Uoj1mV793nKTgroj_0STaykqjd4sQCjk-g,14.00,0.05,0.75,11101.65,12.85,249.40,0.0,8.75,631.70,4482.50,...,1038.25,0.20,1.15,5.85,0.0,51.75,5.80,5.90,23.00,0.45
-NaFbaGRJwN5iqwetBpwJpcJtFjlewDpJqmBg5rBRxU2Kj-Oo0TrMisw3AXt_Rz5ztm9rsFKfKL_4w,9.55,0.10,1.85,11714.20,13.45,103.55,0.0,2.15,2416.40,6380.35,...,1219.55,1.00,2.50,4.65,0.0,17.60,0.45,3.00,8.25,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zdcfDudniVT1JwuE13kWndYcaqNeyzXIKmV-HZ_Oti6CJJtPWKct6uxRtCT02Jbo_1kYbyVD5vo7xw,7.05,0.00,0.70,10047.00,12.20,146.15,0.0,4.35,2689.95,6409.15,...,678.45,0.95,1.75,5.40,0.0,20.85,0.60,3.65,9.95,0.55
zpmR2xbjcpeFSzBsOElGY87O23s0o-yIn0oTbH8-wjCZ0MIGRZFRIdD38uJKixL-EU_Vc1WjEzwhxA,10.75,0.00,1.80,11093.00,13.05,206.40,0.0,3.00,2353.75,4302.65,...,1149.80,0.85,1.90,5.70,0.0,39.40,1.20,6.70,16.80,0.50
zuKQWNAADdb-B5AUtUE5bjOXx8th_IVm2qjbh_MFaGEeXnVW1eScDOtIXPuLHZPxp0-kFua1iHX83A,8.70,0.00,1.90,10645.10,12.75,203.75,0.0,6.05,5163.60,11257.70,...,1120.60,1.60,2.60,5.25,0.0,37.85,4.05,4.55,19.10,0.50


In [15]:
users_in_df = all_users_df['user_id'].unique()
all_rows = []
for index, game in all_users_df.iterrows():
    print(index)
    if not game['in_top_300']:
        continue
    parts = game['participants']
    row = pd.Series(dtype='object')
    other_cols = ml_drop_cols
    for col in other_cols:
        row[col] = game[col]
    for i, p in enumerate(parts):
        if p in users_in_df:
            stats = per_user_stats_df.loc[p, :]
        else:
            stats = per_user_stats_df.loc[users_in_df[0], :]
        new_cols = [col.replace('participant_me_', f'participant_{i}_avg_') for col in per_user_stats_df.columns]
        avg_stats = stats.values
        cur_stats = pd.Series(data=avg_stats, index=new_cols)
        row = pd.concat([row, cur_stats], axis=0)
    all_rows.append(row)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
new_games_df = pd.DataFrame(all_rows)
new_games_df

,matchId,in_top_300,dataVersion,participants,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,...,participant_9_avg_trueDamageTaken,participant_9_avg_turretKills,participant_9_avg_turretTakedowns,participant_9_avg_turretsLost,participant_9_avg_unrealKills,participant_9_avg_visionScore,participant_9_avg_visionWardsBoughtInGame,participant_9_avg_wardsKilled,participant_9_avg_wardsPlaced,participant_9_avg_win
0,NA1_4287589467,True,2,[4n6VLbg0pk_cueIdBdSBhlzIeg7m7G8fj8VfcI8P9EZQy...,1650843457000,1394,1.650845e+12,4287589467,CLASSIC,teambuilder-match-4287589467,...,774.45,0.20,1.30,5.40,0.0,51.55,6.05,6.65,26.10,0.35
1,NA1_4300589612,True,2,[kHdmjAwa4O8cxIuEX9KWTLO2NQhpG7mRjHReJDy1Ya21T...,1651978505000,1566,1.651980e+12,4300589612,CLASSIC,teambuilder-match-4300589612,...,749.60,0.45,1.30,5.40,0.0,49.60,6.90,5.70,26.35,0.45
2,NA1_4290881794,True,2,[cnwEcRHSgzqksmNOr3BYJWFOvQmRo9JndCbZtmGGkxXyQ...,1651113065000,1237,1.651114e+12,4290881794,CLASSIC,teambuilder-match-4290881794,...,749.60,0.45,1.30,5.40,0.0,49.60,6.90,5.70,26.35,0.45
3,NA1_4292866609,True,2,[xNrMpJ3lnm38urzuG0Dlp6VhEOhhjYWUTmrhLJVb2uwGA...,1651287550000,1979,1.651290e+12,4292866609,CLASSIC,teambuilder-match-4292866609,...,479.10,0.55,1.70,4.65,0.0,29.45,0.90,2.75,15.30,0.35
4,NA1_4293694040,True,2,[hco4DhxnUnErXbsgjwKNQmbZdgyE9Bvrt_vPBx_nzMwQZ...,1651366976000,1122,1.651368e+12,4293694040,CLASSIC,teambuilder-match-4293694040,...,926.75,0.85,3.45,4.40,0.0,34.60,0.00,4.75,18.25,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,NA1_4290710811,True,2,[iQnZWXbLW3L3zoqTqxnbkOMc3N-kiMwpN5OB-t12xUv_Z...,1651100971000,1976,1.651103e+12,4290710811,CLASSIC,teambuilder-match-4290710811,...,967.65,0.10,0.90,3.65,0.0,63.30,7.95,8.20,32.50,0.65
5996,NA1_4301905858,True,2,[8s6pxbye--5Zv2JQbaUdZcBUTuVpmmXjHmKS7NYwm5W0q...,1652109034000,1186,1.652110e+12,4301905858,CLASSIC,teambuilder-match-4301905858,...,1004.85,1.60,3.65,4.75,0.0,38.80,2.25,5.05,16.45,0.65
5997,NA1_4291343121,True,2,[a_uG5h8HP90jmv8v2EE5vcXHeTIOTJ7YXZwbtqjCT0-7l...,1651167291000,1780,1.651169e+12,4291343121,CLASSIC,teambuilder-match-4291343121,...,2089.35,2.55,3.90,4.55,0.0,13.15,0.65,2.20,5.55,0.60
5998,NA1_4291276662,True,2,[nGQtB-zBd2lvzEOC7uDViJq1Y8CWgj3lHOwQSe7iR9Y3Y...,1651161277000,1265,1.651163e+12,4291276662,CLASSIC,teambuilder-match-4291276662,...,757.60,2.25,3.65,4.95,0.0,20.75,2.25,5.40,8.65,0.65


In [17]:
new_games_df.to_csv(Path('parsed_data.csv'), index=False)

In [11]:
ml_drop_cols = ['matchId', 'dataVersion', 'participants', 'gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'platformId', 'queueId', 'tournamentCode']           